In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from nltk.tokenize import TweetTokenizer
from collections import defaultdict

In [ ]:
df = pd.read_csv("sentiment_analysis.csv", index_col="id")

df.head()

In [ ]:
def text_normalize(text):
    # Retweet old acronym "RT" removal
    text = re.sub(r"^RT[\s]+", "", text)

    # Hyperlinks removal
    text = re.sub(r"https?://.*[\r\n]*", "", text)

    # Hashtags removal
    text = re.sub(r"#", "", text)

    # Punctuation removal
    text = re.sub(r"[^\w\s]", "", text)

    # Tokenization
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    text_tokens = tokenizer.tokenize(text)

    return text_tokens


def get_freqs(df):
    freqs = defaultdict(lambda: 0)
    for idx, row in df.iterrows():
        tweet = row["tweet"]
        label = row["label"]
        tokens = text_normalize(tweet)
        for token in tokens:
            pair = (token, label)
            freqs[pair] += 1
    return freqs


def get_feature(text, freqs):
    tokens = text_normalize(text)
    X = np.zeros(3)
    X[0] = 1
    for token in tokens:
        X[1] += freqs[(token, 0)]
        X[2] += freqs[(token, 1)]
    return X

In [ ]:
X = []
y = []

freqs = get_freqs(df)
for idx, row in df.iterrows():
    tweet = row["tweet"]
    label = row["label"]

    X_i = get_feature(tweet, freqs)
    X.append(X_i)
    y.append(label)

X = np.array(X)
y = np.array(y)

In [ ]:
val_size = 0.2
test_size = 0.125
random_state = 2
is_shuffle = True

# First split: train+val and test
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=test_size, random_state=random_state, shuffle=is_shuffle
)

# Second split: train and val
val_ratio = val_size / (1 - test_size)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=val_ratio, random_state=random_state, shuffle=is_shuffle
)

In [ ]:
normalizer = StandardScaler()
X_train[:, 1:] = normalizer.fit_transform(X_train[:, 1:])
X_val[:, 1:] = normalizer.transform(X_val[:, 1:])
X_test[:, 1:] = normalizer.transform(X_test[:, 1:])

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))


def compute_loss(y_hat, y):
    # Đảm bảo y_hat nằm trong khoảng [1e-7, 1 - 1e-7] để tránh log(0)
    y_hat = np.clip(y_hat, 1e-7, 1 - 1e-7)
    return (-y * np.log(y_hat) - (1 - y) * np.log(1 - y_hat)).mean()


def predict(X, theta):
    dot_product = np.dot(X, theta)
    return sigmoid(dot_product)


def compute_gradient(X, y, y_hat):
    return np.dot(X.T, (y_hat - y)) / y.size


def update_theta(theta, gradient, lr):
    return theta - lr * gradient


def compute_accuracy(y_hat, y):
    return np.mean(np.round(y_hat) == y)

In [ ]:
lr = 0.01
epochs = 200
batch_size = 128

np.random.seed(random_state)
theta = np.random.uniform(size=X_train.shape[1])


train_accs = []
train_losses = []
val_accs = []
val_losses = []


for epoch in range(epochs):
    train_batch_losses = []
    train_batch_accuracies = []
    val_batch_losses = []
    val_batch_accuracies = []

    for i in range(0, X_train.shape[0], batch_size):
        X_batch = X_train[i : i + batch_size]
        y_batch = y_train[i : i + batch_size]

        y_hat = predict(X_batch, theta)
        loss = compute_loss(y_hat, y_batch)
        train_batch_losses.append(loss)

        accuracy = compute_accuracy(y_hat, y_batch)
        train_batch_accuracies.append(accuracy)

        gradient = compute_gradient(X_batch, y_batch, y_hat)
        theta = update_theta(theta, gradient, lr)

        # Validation
        y_val_hat = predict(X_val, theta)
        val_loss = compute_loss(y_val_hat, y_val)
        val_batch_losses.append(val_loss)
        val_accuracy = compute_accuracy(y_val_hat, y_val)
        val_batch_accuracies.append(val_accuracy)

    train_losses.append(np.mean(train_batch_losses))
    train_accs.append(np.mean(train_batch_accuracies))
    val_losses.append(np.mean(val_batch_losses))
    val_accs.append(np.mean(val_batch_accuracies))

    print(
        f"Epoch {epoch + 1}/{epochs} - "
        f"Train Loss: {train_losses[-1]:.4f}, "
        f"Train Accuracy: {train_accs[-1]:.4f}, "
        f"Validation Loss: {val_losses[-1]:.4f}, "
        f"Validation Accuracy: {val_accs[-1]:.4f}"
    )

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12, 10))

ax[0, 0].plot(train_losses)
ax[0, 0].set(xlabel="Epoch", ylabel="Loss")
ax[0, 0].set_title("Training Loss")

ax[0, 1].plot(val_losses, color="orange")
ax[0, 1].set(xlabel="Epoch", ylabel="Loss")
ax[0, 1].set_title("Validation Loss")

ax[1, 0].plot(train_accs)
ax[1, 0].set(xlabel="Epoch", ylabel="Accuracy")
ax[1, 0].set_title("Training Accuracy")

ax[1, 1].plot(val_accs, color="orange")
ax[1, 1].set(xlabel="Epoch", ylabel="Accuracy")
ax[1, 1].set_title("Validation Accuracy")

plt.show()

In [ ]:
val_set_acc = compute_accuracy(predict(X_val, theta), y_val)
test_set_acc = compute_accuracy(predict(X_test, theta), y_test)
print("Evaluation on validation and test set:")
print(f"Validation Accuracy: {val_set_acc}")
print(f"Test Accuracy: {test_set_acc}")